In [1]:
import os
os.chdir('../..')

In [2]:
import sys
import pandas as pd

import pickle

import matplotlib.pyplot as plot

from os import listdir
from os.path import isfile, join

import numpy as np

import lightgbm as lgb

import glob

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from modules.ml_efficacy.LGBMOrdinal_base import LGBMOrdinal, LGBMRegressor, LGBMClassifier, LightGBMCV, emse, emae

import warnings
warnings.filterwarnings('ignore')

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Functions

In [3]:
def process_real_df(df, cat_cols=None, ord_cols=None, cont_cols=None, enc=None):
    if cat_cols and cont_cols:
        df[cat_cols+ord_cols] = enc.fit_transform(df[cat_cols+ord_cols])
    else:
        print('Automated inference of column types to be implemented!')
    return df

def process_syn_df(df, cat_cols, ord_cols, cont_cols, enc=None):
    df[cat_cols+ord_cols] = enc.transform(df[cat_cols+ord_cols])

    return df

def is_a_DATGAN(name):
    if any(x in name for x in ['TGAN', 'CTGAN', 'FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']):
        return False
    else:
        return True
    
def check_low_appearing_vars(df):
    
    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print('Variable {}: '.format(c))
                for idx, v in zip(val.index, val):
                    if v < 0.01:
                        print('  {} - {:.2f}% ({:d})'.format(idx, 100*v, int(v*len(df))))
                print()
                
def replace_low_appearing_values(df, dataset):
    
    if 'Chicago' in dataset:
        dct_ = {}
        for i in df['hh_vehicles'].unique():
            if i >= 5:
                dct_[i] = '5+'
            else:
                dct_[i] = str(i)        
        df['hh_vehicles'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_size'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_size'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_bikes'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_bikes'].replace(dct_, inplace=True)       

    elif 'LPMC' in dataset:
        dct_ = {}
        for i in df['pt_n_interchanges'].unique():
            if i >= 2:
                dct_[i] = '2+'
            else:
                dct_[i] = str(i)        
        df['pt_n_interchanges'].replace(dct_, inplace=True) 
        
        dct_ = {
            'Diesel_LGV': 'LGV',
            'Petrol_LGV': 'LGV',
            'Hybrid_Car': 'Average_Car'
        }

# Get all models and associated files

In [4]:
dataset = 'Chicago_DAG'
input_folder = '../synth_data/{}/'.format(dataset)
n_models = 5
n_data = 5

# Models for testing all DATGANS
models = ['CTGAN', 'TGAN']

for i in ['WGAN', 'SGAN', 'WGGP']:
    for j in ['WI', 'OR', 'WO']:
        for k in ['NO', 'BO', 'OD']:
            models.append('{}_{}_{}'.format(i,j,k))
            
# Models for testing different DAGs
models = ['FULL', 'TRANSRED', 'LINEAR', 'NOLINKS', 'PREDICTION']

models.sort()

files_ = {}

for m in models:
    tmp = []
    if is_a_DATGAN(m):
        spl = m.split('_')
        for i in range(n_models):
            for j in range(n_data):
                tmp.append(input_folder + '{}_{}_{:0>2}_{}_{:0>2}.csv'.format(spl[0], spl[1], i+1,  spl[2], j+1))
    else:
        for i in range(n_models):
            for j in range(n_data):
                tmp.append(input_folder + '{}_{:0>2}_{:0>2}.csv'.format(m, i+1, j+1))
    files_[m] = tmp


models.append('original')

files_['original'] = ['../data/' + dataset.split('_')[0] + '/data.csv' ]

In [5]:
df_orig = pd.read_csv('../data/' + dataset.split('_')[0] + '/data.csv')

In [6]:
df_orig.columns

Index(['choice', 'travel_dow', 'trip_purpose', 'distance', 'hh_vehicles',
       'hh_size', 'hh_bikes', 'hh_descr', 'hh_income', 'gender', 'age',
       'license', 'education_level', 'work_status', 'departure_time'],
      dtype='object')

In [7]:
replace_low_appearing_values(df_orig, dataset)

In [8]:
check_low_appearing_vars(df_orig)

In [9]:
if 'Chicago' in dataset:
    cont_cols = ['distance', 'age', 'departure_time']
    ord_cols = ['hh_vehicles', 'hh_size', 'hh_bikes', 'hh_income', 
                'education_level']
    cat_cols = [col for col in df_orig.columns if col not in cont_cols + ord_cols]
elif 'LPMC' in dataset:
    cont_cols = ['start_time_linear', 'age', 'distance', 'dur_walking', 
                 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 
                 'dur_pt_int', 'dur_driving', 'cost_transit', 
                 'cost_driving_fuel', 'driving_traffic_percent']
    ord_cols = ['survey_year', 'travel_year', 'travel_month', 'travel_date', 
                'day_of_week', 'pt_n_interchanges', 'car_ownership']
    cat_cols = [col for col in df_orig.columns if col not in cont_cols + ord_cols]

In [10]:
enc = OrdinalEncoder()
ori = process_real_df(df_orig, cat_cols, ord_cols, cont_cols, enc)

# Generate results

In [11]:
filepath = './notebooks/results/{}/'.format(dataset)
filename = 'cv_result_ml.pickle'
cv_modelscores = {}
params={'n_estimators': 5000}

try:
    cv_modelscores = pickle.load(open(f'{filepath}{filename}','rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

Found previous pickel file, using that


In [12]:
for i, model in enumerate(models):
    
    if model in cv_modelscores.keys():
        print("Results for model \033[1m{}\033[0m ({}/{}) already exists".format(model, i+1, len(models)))
    else:
        print("Getting results for model \033[1m{}\033[0m ({}/{})".format(model, i+1, len(models)))
        cv_modelscores[model] = []
        
    n_files_done = len(cv_modelscores[model])
        
    for j, f in enumerate(files_[model][n_files_done:]):
        
        print("  Processing file {}/{}".format(j+1+n_files_done, len(files_[model])) + " "*20)
        
        tmp_df = pd.read_csv(f)
        replace_low_appearing_values(tmp_df, dataset)
        v_df = process_syn_df(tmp_df, cat_cols, ord_cols, cont_cols, enc)
        
        tmp = {}
        for k, ycol in enumerate(ori.columns):
            info = '    Column: {} ({}/{})'.format(ycol, k+1, len(ori.columns))
            print(info, end="")
            sys.stdout.flush()
            Xcols = [c for c in ori.columns if c!=ycol]

            y_synth = v_df[ycol]
            X_synth = v_df[Xcols]
            y_real = ori[ycol]
            X_real = ori[Xcols]


            observe_sets = {'original': (X_real, y_real)}
            ccols = [c for c in cat_cols if c!=ycol]


            if ycol in cat_cols + ord_cols:
                lgbm_type = 'LGBMClassifier'
                kf = StratifiedKFold(shuffle=True, random_state=42)
                eval_metric = ['error']
            elif ycol in cont_cols:
                lgbm_type = 'LGBMRegressor'
                kf = KFold(shuffle=True, random_state=42)
                eval_metric = ['l2', 'l1']
            cv = LightGBMCV(lgbm_type=lgbm_type,
                splitter = kf,
                eval_metric = eval_metric,
                observe_sets = observe_sets,
                separate_observation_split = True,
                early_stopping_rounds = 5,
                return_cv_models = False,
                refit_model = False,
                verbose = True)
            cv.fit(X_synth, y_synth, categorical_feature=ccols, params=params)
            tmp[ycol] = cv.result_dict_
            
            print(' '*len(info), end='\r')

            if k == len(ori.columns):
                print('', end='\r')
            
        cv_modelscores[model].append(tmp)
            
        pickle.dump(cv_modelscores,open(f'{filepath}/{filename}','wb'))

print("\033[1mFINISHED!\033[0m")

Results for model FULL (1/6) already exists
Results for model LINEAR (2/6) already exists
Results for model NOLINKS (3/6) already exists
Results for model PREDICTION (4/6) already exists
Results for model TRANSRED (5/6) already exists
Results for model original (6/6) already exists
FINISHED!


In [13]:
ori_scores = {col: cv_modelscores['original'][0][col]['test_log_loss'] for col in cat_cols + ord_cols}
ori_scores.update({col: cv_modelscores['original'][0][col]['test_l2'] for col in cont_cols})

internal = {}
external = {}
external_normalised = {}
cont_scores = {}
cat_scores = {}

for model in models:
    
    n_tests = len(cv_modelscores[model])
    
    internal[model] = {}
    external[model] = {}
    external_normalised[model] = {}
    for col in cat_cols + ord_cols:
        tmp = [cv_modelscores[model][i][col]['test_log_loss'] for i in range(n_tests)]
        internal[model][col] = {'avg': np.mean(tmp), 'std': np.std(tmp)}
        
        tmp = [cv_modelscores[model][i][col]['original_log_loss'] for i in range(n_tests)]
        external[model][col] = {'avg': np.mean(tmp), 'std': np.std(tmp)}
        
        external_normalised[model][col] = external[model][col]['avg'] - ori_scores[col]

        
    for col in cont_cols:
        tmp = [cv_modelscores[model][i][col]['test_l2'] for i in range(n_tests)]
        internal[model][col] = {'avg': np.mean(tmp), 'std': np.std(tmp)}
        
        tmp = [cv_modelscores[model][i][col]['original_l2'] for i in range(n_tests)]
        external[model][col] = {'avg': np.mean(tmp), 'std': np.std(tmp)}
        
        external_normalised[model][col] = external[model][col]['avg'] - ori_scores[col]
    
    cont_scores[model] = sum([external[model][col]['avg']/ori_scores[col] for col in cont_cols])
    cat_scores[model] = sum([external[model][col]['avg']-ori_scores[col] for col in cat_cols + ord_cols])

In [16]:
cat_sorted = sorted(cat_scores.items(), key=lambda item: item[1])
cont_sorted = sorted(cont_scores.items(), key=lambda item: item[1])

In [17]:
i=1
print('   | {:<30} | {:<30}'.format('categorical', 'continuous'))
print('-----------------------------------------------------------')
for a, b in zip(cat_sorted, cont_sorted):
    print('{:>2} | {:<30} | {:<30}'.format(i, '{:<12}: {:.3f}'.format(a[0], a[1]), '{:<12}: {:.3f}'.format(b[0], b[1])))
    i+=1

   | categorical                    | continuous                    
-----------------------------------------------------------
 1 | original    : -2.111           | original    : 2.559           
 2 | FULL        : 0.869            | FULL        : 3.133           
 3 | TRANSRED    : 0.917            | TRANSRED    : 3.135           
 4 | LINEAR      : 1.114            | LINEAR      : 3.199           
 5 | PREDICTION  : 3.618            | PREDICTION  : 6.077           
 6 | NOLINKS     : 3.949            | NOLINKS     : 6.220           
